In [1]:
# coding: utf-8

import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import re
import progressbar
from itertools import chain
from multiprocessing import Pool
import functools
import csv
import requests
from collections import defaultdict
import json
import argparse

def get_session(shop):
    source = urllib.request.urlopen("http://www.dfshop.com/TeamShop/")
    soup = bs.BeautifulSoup(source, 'lxml', parse_only=bs.SoupStrainer('td'))

    g = [i[0]+1 for i in enumerate(soup) if re.match(".+"+str(shop)+".+", str(i[1]))]
    link = [i[1] for i in enumerate(soup) if i[0] == g[0]]
    html = [i['href'] for i in link[0].find_all("a")]

    s = requests.Session()
    r = s.get(html[0])
    soup = bs.BeautifulSoup(r.text, "lxml")
    html = ["http://www.dfshop.com"+i['href'] for i in soup.find_all("a")]
    return html

def post_request(session, page):
    s = requests.Session()
    r = s.get(session[0]+"&f=search&txtArnum="+str(page))
    soup = bs.BeautifulSoup(r.text, "lxml")
    return soup

def clean_article(article):
    article = re.sub("\D+", "", article)
    return article

def clean_price(price):
    price = re.sub("[^0-9.]", "", price)
    return float(price)

def next_page(session):
    s = requests.Session()
    r = s.get(session+"&f=listpage&nav=next")
    soup = bs.BeautifulSoup(r.text, "lxml")
    return soup

def loop_page(session, page):
    l = {}
    pages = []
    from_ = 0
    to_ = 1
    while from_ != to_:
        try:
            if from_ == 0:
                s = post_request(session, page)
            else:
                s = next_page(session)
            page_string = [re.findall("Seite.\d+\svon\s\d+", i.text)[0] for i in s.select("#artListFooter")]
            page = re.findall("\d+", page_string[0])
            from_, to_ = page[0], page[1]
            pages.append([from_, to_])

            article = s.select(".artdesc")
            dimension = s.select(".artdimension")
            price = s.select(".artprice")

            for x, y, z in zip(article, dimension, price):
                artnumbr = re.findall("art=\w{2}\d+", str(x))
                text = re.findall("art=.+>.+<", str(x))
                text2 = re.findall("arttext.+<", str(x))
                price_ = re.findall("artprice.+>\d+.+<", str(z))
                try:
                    artnumbr = clean_article(" ".join(artnumbr))
                    #text = text[0]
                    text3 = " ".join(text2)
                    price_ = clean_price(" ".join(price_))
                    l[artnumbr] = [text2, price_]
                except:
                    continue
        except:
            continue
    return l


def parse_shop(shop, catalog=(0,10)):
    session = get_session(shop)
    container = defaultdict(lambda: defaultdict(list))

    for p in range(catalog[0], catalog[1]):
        print("Parsing Catalog Number: %s" % (str(p)))
        container[shop][str(p)] = loop_page(session, p)

    return container

In [2]:
session = get_session("Sabag")[0]
session

'http://www.dfshop.com/dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG'

In [3]:
def get_category(session):
    s = requests.Session()
    r = s.get(session)
    soup = bs.BeautifulSoup(r.text, 'lxml', parse_only=bs.SoupStrainer('a'))
    links = set()
    for i in soup.find_all("a", href = re.compile(r'.+cat=\d.+')):
        links.add("http://www.dfshop.com/"+i['href'])
    links = list(links)
    links.sort()
    return links

In [4]:
cats = get_category(session)
cats

['http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=10.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=2.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=3.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=4.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=5.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=6.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=7.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.p?session=AOkibvRdiRacUacslgchdAUikeUGolRG&f=cat&cat=8.0.0.0',
 'http://www.dfshop.com//dfshop/wsTeamShop.wsc/pTsHome.

In [5]:
def next_page_by_cat(session):
    s = requests.Session()
    session = re.sub("&f=cat&cat=\d.+", "&f=listpage&nav=next", session)
    r = s.get(session)
    soup = bs.BeautifulSoup(r.text, "lxml")
    return soup

In [6]:
def init_session(session):
    s = requests.Session()
    r = s.get(session)
    #return print("Session Initialized")

In [7]:
def loop_article_href(session):
    init_session(session)
    links = set()
    from_, to_ = int(0), int(1)
    while to_ > from_:
        try:
            soup = next_page_by_cat(session)

            for i in soup.find_all('a', href = re.compile(r'.+art=\w{2}\d+')):
                links.add(re.findall(r'art=\w{2}\d+',i['href'])[0][4:])

            indexes = [re.findall(r'\d+', i.text) for i in soup.find_all('div',{"id": "artListFooter"})][0]

            from_ = int(indexes[1])
            to_ = int(indexes[2])
            
            percent = float(indexes[1]) / float(indexes[2])
            
            if round(percent,1) in range(10, 110, 10):
                print("Parsing: %s of %s (%s)" % (str(indexes[1]), str(indexes[2]), str(round(percent,1))))
        except requests.exceptions.HTTPError as err:
            print(err)
            continue
    
    links = list(links)
    links.sort()
    return links

In [8]:
def get_all_article_links(categorys):
    
    container = {}
    
    for c in categorys:
        cat_string = str(re.findall(r'(cat=\d+\.\d+\.\d+\.\d+)', c)[0])
        print("Parsing: %s" % (re.sub("=", " = ", cat_string)))
        container[cat_string] = loop_article_href(c)
        
    return container

In [ ]:
article_refs = get_all_article_links(cats)

Parsing: cat = 10.0.0.0


In [9]:
def output_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    
    return print("Writing %s" % (str(filename)))

In [33]:
output_json_file(article_refs, "test.json")

Writing test.json


In [11]:
def read_json_file(filename):
    with open(filename) as data_file:    
        data = json.load(data_file)
    return data

In [12]:
d = read_json_file("test.json")

In [14]:
def parse_article(session, idArticle):
    container = {}
    for article in idArticle.values():
        for i in article:
            s = requests.Session()
            r = s.get(session + "&f=art&art=" + str(i))
            
            soup = bs.BeautifulSoup(r.text, 'lxml', parse_only=bs.SoupStrainer('div', {"id": "artDetailMain"}))
            text = [re.sub("\n", "", i.text) for i in soup.find_all("div", {'id': 'artDetailDescRow'})]
            idsupplier = [re.sub("\n", "", i.text) for i in soup.find_all("div", {'id': 'artDetailLiefArnumRow'})]
            price = [re.findall("\d+\.\d+", i.text)[0] for i in soup.find_all("div", {'id': 'artDetailLiefPriceRow'})]
            
            container[str(i)] = [text, idsupplier, price]        
    return container

In [15]:
g = parse_article(session, d)

IndexError: list index out of range

In [120]:
g

{'AA014671': [['BeschreibungTauchrohr TECEdrainlinezu Ablauf flach 146 771'],
  ['Artikel-Nr des Herstellers6.680.10'],
  ['3.20']],
 'AA014672': [['BeschreibungTauchrohr TECEdrainlinezu Ablauf Norm 146 772'],
  ['Artikel-Nr des Herstellers6.680.11'],
  ['3.20']],
 'AA014673': [['BeschreibungTauchrohr TECEdrainlinezu Ablauf max und senkrecht146 773/774'],
  ['Artikel-Nr des Herstellers6.680.19'],
  ['3.60']],
 'AA014674': [['BeschreibungTauchrohr TECEdrainlinezu Ablauf superflach 146 770'],
  ['Artikel-Nr des Herstellers6.680.25'],
  ['3.20']],
 'AA014675': [['BeschreibungMembran-Geruchsverschluss fürAblauf TECEdrainlinesuperflach 146 770'],
  ['Artikel-Nr des Herstellers6.600.15'],
  ['21.00']],
 'AA014676': [['BeschreibungMembran-Geruchsverschluss fürAblauf TECEdrainline super-flach 146 771'],
  ['Artikel-Nr des Herstellers6.600.16'],
  ['22.50']],
 'AA014677': [['BeschreibungMembran-Geruchsverschluss fürAblauf TECEdrainline Norm146 772'],
  ['Artikel-Nr des Herstellers6.600.18'],
  